In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 10.0 MB/s eta 0:00:00


In [2]:
# import necessary library
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source


import pandas as pd

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.shape

(768, 9)

In [4]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [5]:
# spilit the dataset into features (X) and target (y)
x = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


### Samplers in Optuna

#### Using TPE Sampler (By default)

 TPE sampler which uses Bayesian Optimization
    **** Uses probability models (like Gaussian Processes) to smartly explore the search space. ****


In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# define the objective function

def objective(trial):
    # define the hyperparameters to be optimized
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)


    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
    accuracy = scores.mean()

    return accuracy # Return the accuracy score for Optuna to maximize



In [25]:
# Create a study object and optimize the objective function

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-09-10 18:56:12,856] A new study created in memory with name: no-name-805388c9-5cc0-4f4f-aef9-e7d455ec6b55
[I 2025-09-10 18:56:13,880] Trial 0 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 187, 'max_depth': 4}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-09-10 18:56:14,957] Trial 1 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 181, 'max_depth': 13}. Best is trial 1 with value: 0.7728119180633147.
[I 2025-09-10 18:56:15,301] Trial 2 finished with value: 0.7541899441340782 and parameters: {'n_estimators': 66, 'max_depth': 3}. Best is trial 1 with value: 0.7728119180633147.
[I 2025-09-10 18:56:16,120] Trial 3 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 144, 'max_depth': 7}. Best is trial 1 with value: 0.7728119180633147.
[I 2025-09-10 18:56:17,299] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 200, 'max_depth': 15}. Best is trial 1 with value: 0.77281191

In [26]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 117, 'max_depth': 15}


In [27]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


#### Using Random Search optuna

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [11]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-09-10 18:33:03,611] A new study created in memory with name: no-name-86be0a8a-d90d-419b-aa7a-0bd61bea21ed
[I 2025-09-10 18:33:04,633] Trial 0 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 191, 'max_depth': 3}. Best is trial 0 with value: 0.7579143389199254.
[I 2025-09-10 18:33:05,671] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 178, 'max_depth': 15}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-09-10 18:33:06,519] Trial 2 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 144, 'max_depth': 16}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-09-10 18:33:07,184] Trial 3 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 114, 'max_depth': 12}. Best is trial 3 with value: 0.7746741154562384.
[I 2025-09-10 18:33:08,113] Trial 4 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 188, 'max_depth': 3}. Best is trial 3 with value: 0.774674

In [12]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 133, 'max_depth': 16}


In [13]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


#### Using Grid Search

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
  # suggest the value for the hyperparameters
  n_estimators = trial.suggest_int('n_estimators', 50, 200)
  max_depth = trial.suggest_int('max_depth', 3, 20)


  # create a RandomForestClassifier with suggested hyperparameters
  model = RandomForestClassifier(
      n_estimators=n_estimators,
      max_depth=max_depth,
      random_state=42
  )

  # perform 3-fold cross-validation and calculate accuracy
  score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

  return score  # Return the accuracy score for Optuna to maximize

In [18]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [21]:
study = optuna.create_study(direction = 'maximize' , sampler = optuna.samplers.GridSampler(search_space=search_space))
study.optimize(objective)
# study.optimize(objective, n_trials = 0 < x <= len(n_estimeters) * len(max_depth))

[I 2025-09-10 18:46:42,756] A new study created in memory with name: no-name-c5922ad1-2535-4c86-8202-3137ffda018d
[I 2025-09-10 18:46:43,301] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-10 18:46:44,169] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-10 18:46:44,487] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-10 18:46:45,070] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-10 18:46:45,695] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [22]:
# print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [23]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


### Optuna visualization

In [28]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [29]:
# 1. Optimization History
plot_optimization_history(study).show()

In [30]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 4. Contour Plot
plot_contour(study).show()

In [33]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

### Optimizing Multiple ML Models

just like hyperparameters we also don't know which model should we use for prediction so we will use different models as a hyperparameter  

In [34]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [35]:
# Define the objective function for Optuna

def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [36]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-09-10 19:12:28,297] A new study created in memory with name: no-name-5597e60b-2087-4de9-bcdd-e84019774f91
[I 2025-09-10 19:12:29,910] Trial 0 finished with value: 0.7635009310986964 and parameters: {'classifier': 'RandomForest', 'n_estimators': 164, 'max_depth': 17, 'min_samples_split': 7, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 0 with value: 0.7635009310986964.
[I 2025-09-10 19:12:29,955] Trial 1 finished with value: 0.7653631284916201 and parameters: {'classifier': 'SVM', 'C': 6.328160476239065, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 1 with value: 0.7653631284916201.
[I 2025-09-10 19:12:29,992] Trial 2 finished with value: 0.7635009310986964 and parameters: {'classifier': 'SVM', 'C': 0.2730849514689149, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 1 with value: 0.7653631284916201.
[I 2025-09-10 19:12:30,025] Trial 3 finished with value: 0.7262569832402234 and parameters: {'classifier': 'SVM', 'C': 1.5901743405871125, 'kernel': 'poly', 'gamma

In [37]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.11540311414178571, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [39]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.763501,2025-09-10 19:12:28.301688,2025-09-10 19:12:29.910181,0 days 00:00:01.608493,NaN,True,RandomForest,NaN,NaN,NaN,17.0,4.0,7.0,164.0,COMPLETE
1,1,0.765363,2025-09-10 19:12:29.911304,2025-09-10 19:12:29.955215,0 days 00:00:00.043911,6.328160,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.763501,2025-09-10 19:12:29.956329,2025-09-10 19:12:29.992601,0 days 00:00:00.036272,0.273085,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.726257,2025-09-10 19:12:29.993675,2025-09-10 19:12:30.025316,0 days 00:00:00.031641,1.590174,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.785847,2025-09-10 19:12:30.026501,2025-09-10 19:12:30.093368,0 days 00:00:00.066867,9.004002,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.789572,2025-09-10 19:13:13.895812,2025-09-10 19:13:13.929427,0 days 00:00:00.033615,0.122502,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.716946,2025-09-10 19:13:13.930498,2025-09-10 19:13:13.964280,0 days 00:00:00.033782,0.196666,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.789572,2025-09-10 19:13:13.965341,2025-09-10 19:13:13.998235,0 days 00:00:00.032894,0.151095,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2025-09-10 19:13:13.999568,2025-09-10 19:13:14.037946,0 days 00:00:00.038378,0.269905,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [42]:
# we can see that how many times each model were used
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,79
RandomForest,12
GradientBoosting,9


In [43]:
# we can see the average accuracy of all the model used

study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.750259
RandomForest,0.768001
SVM,0.776018


In [44]:
# 1. Optimization History
plot_optimization_history(study).show()

In [45]:
# 3. Slice Plot
plot_slice(study).show()

In [46]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [50]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 5.0 MB/s eta 0:00:00


In [51]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-09-10 19:22:35,291] A new study created in memory with name: no-name-1929f225-d74a-4e02-98b8-007dcb5f1011


[0]	train-mlogloss:1.04763	eval-mlogloss:1.04349
[1]	train-mlogloss:0.97470	eval-mlogloss:0.96751
[2]	train-mlogloss:0.90901	eval-mlogloss:0.90025
[3]	train-mlogloss:0.84801	eval-mlogloss:0.83797
[4]	train-mlogloss:0.79352	eval-mlogloss:0.78120
[5]	train-mlogloss:0.74269	eval-mlogloss:0.72620
[6]	train-mlogloss:0.69769	eval-mlogloss:0.67925
[7]	train-mlogloss:0.65650	eval-mlogloss:0.63691
[8]	train-mlogloss:0.61800	eval-mlogloss:0.59538
[9]	train-mlogloss:0.58272	eval-mlogloss:0.55803
[10]	train-mlogloss:0.55031	eval-mlogloss:0.52420
[11]	train-mlogloss:0.52070	eval-mlogloss:0.49274
[12]	train-mlogloss:0.50360	eval-mlogloss:0.47595
[13]	train-mlogloss:0.47717	eval-mlogloss:0.44688
[14]	train-mlogloss:0.46160	eval-mlogloss:0.43042
[15]	train-mlogloss:0.44120	eval-mlogloss:0.40761
[16]	train-mlogloss:0.42240	eval-mlogloss:0.38776
[17]	train-mlogloss:0.40676	eval-mlogloss:0.37174
[18]	train-mlogloss:0.38948	eval-mlogloss:0.35449
[19]	train-mlogloss:0.37631	eval-mlogloss:0.34002
[20]	train

[I 2025-09-10 19:22:42,062] Trial 0 finished with value: 1.0 and parameters: {'lambda': 6.080062247534038e-05, 'alpha': 0.00035874977849511614, 'eta': 0.062106874141135214, 'gamma': 0.0022812895229936763, 'max_depth': 7, 'min_child_weight': 9, 'subsample': 0.8436934864370882, 'colsample_bytree': 0.6001493436899629}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.97132	eval-mlogloss:0.97598
[1]	train-mlogloss:0.81567	eval-mlogloss:0.81334
[2]	train-mlogloss:0.69425	eval-mlogloss:0.68335
[3]	train-mlogloss:0.59664	eval-mlogloss:0.58346
[4]	train-mlogloss:0.51690	eval-mlogloss:0.50005
[5]	train-mlogloss:0.45128	eval-mlogloss:0.43046
[6]	train-mlogloss:0.39707	eval-mlogloss:0.37402
[7]	train-mlogloss:0.35207	eval-mlogloss:0.32495
[8]	train-mlogloss:0.31440	eval-mlogloss:0.28323
[9]	train-mlogloss:0.28629	eval-mlogloss:0.25044
[10]	train-mlogloss:0.26104	eval-mlogloss:0.22302
[11]	train-mlogloss:0.24052	eval-mlogloss:0.19786
[12]	train-mlogloss:0.22996	eval-mlogloss:0.18577
[13]	train-mlogloss:0.21461	eval-mlogloss:0.16747
[14]	train-mlogloss:0.20571	eval-mlogloss:0.15675
[15]	train-mlogloss:0.19711	eval-mlogloss:0.14572
[16]	train-mlogloss:0.19289	eval-mlogloss:0.14094
[17]	train-mlogloss:0.18900	eval-mlogloss:0.13662
[18]	train-mlogloss:0.18531	eval-mlogloss:0.13237
[19]	train-mlogloss:0.18410	eval-mlogloss:0.13254
[20]	train

[I 2025-09-10 19:22:48,606] Trial 1 finished with value: 1.0 and parameters: {'lambda': 2.2186923211427153e-08, 'alpha': 0.006210324509578505, 'eta': 0.15011715095251338, 'gamma': 4.386862718722596e-07, 'max_depth': 3, 'min_child_weight': 8, 'subsample': 0.9828141994495608, 'colsample_bytree': 0.573074828467165}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.95661	eval-mlogloss:0.94739
[1]	train-mlogloss:0.78032	eval-mlogloss:0.75725
[2]	train-mlogloss:0.64815	eval-mlogloss:0.61512


[I 2025-09-10 19:22:58,915] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:0.91967	eval-mlogloss:0.91095
[1]	train-mlogloss:0.78096	eval-mlogloss:0.76481


[I 2025-09-10 19:22:58,956] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 6.080062247534038e-05, 'alpha': 0.00035874977849511614, 'eta': 0.062106874141135214, 'gamma': 0.0022812895229936763, 'max_depth': 7, 'min_child_weight': 9, 'subsample': 0.8436934864370882, 'colsample_bytree': 0.6001493436899629}
Best accuracy: 1.0


In [52]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()